In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import textwrap
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [3]:
df = pd.read_csv('RussianOligarchPropertyMap.csv')

In [4]:
df.columns

Index(['Still Own?', 'Sale Date', 'Name', 'Oligarch_Description',
       'Full_Address', 'City', 'Sale Price', 'Property Description',
       'Property Type', 'Story/Reference Link', 'Story/Reference Link #2'],
      dtype='object')

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['Full_Address'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

0     (7600 Fisher Island Dr APT 7642, Miami Beach, ...
1     (101 NW 20th St, Miami, FL 33127, USA, (25.795...
2     (525 N County Rd, Palm Beach, FL 33480, USA, (...
3     (111 W 57th St, New York, NY 10019, USA, (40.7...
4     (111 W 57th St, New York, NY 10019, USA, (40.7...
5     (260 5th Ave, New York, NY 10001, USA, (40.745...
6     (9 E 75th St, New York, NY 10021, USA, (40.774...
7     (15 E 75th St, New York, NY 10021, USA, (40.77...
8     (215 E 75th St, New York, NY 10021, USA, (40.7...
9     (11 E 64th St, New York, NY 10065, USA, (40.76...
10    (12 Gay St, New York, NY 10014, USA, (40.73360...
11    (2 E 63rd St, New York, NY 10065, USA, (40.766...
12    (15 E 64th St, New York, NY 10065, USA, (40.76...
13    (834 Fifth Avenue, New York, NY 10065, USA, (4...
14    (19 E 64th St, New York, NY 10065, USA, (40.76...
15    (998 5th Ave, New York, NY 10028, USA, (40.778...
16    (33 E 74th St, New York, NY 10021, USA, (40.77...
17    (785 5th Ave, New York, NY 10022, USA, (40

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,Still Own?,Sale Date,Name,Oligarch_Description,Full_Address,City,Sale Price,Property Description,Property Type,Story/Reference Link,Story/Reference Link #2,loc,point,lat,lon,altitude
0,No,December 2018,Aras Agalarov,Founded one of Russia's leading development fi...,"7600 Fisher Island unit 7642, Miami Florida 33...",Miami,$8.46M,Fisher Island Condo,Residential,https://therealdeal.com/miami/2018/12/03/azerb...,NaN,"(7600 Fisher Island Dr APT 7642, Miami Beach, ...","(25.7612272, -80.1363805, 0.0)",25.761227,-80.136381,0.0
1,No,May 2018,Oleg Babibakov,A mining Oligarch who is co-developing Missoni...,"101 20th Street, Miami, Florida",Miami,$9.53M,Setai Penthouse,Residential,https://therealdeal.com/miami/2018/05/30/russi...,https://therealdeal.com/miami/2017/05/19/oleg-...,"(101 NW 20th St, Miami, FL 33127, USA, (25.795...","(25.7955326, -80.1992159, 0.0)",25.795533,-80.199216,0.0
2,No,July 2019,Dmitry Rybolovlev,In 2010 Ryboloblev was reported to be the 10th...,"525 North County Road, Palm Beach, Florida",Palm Beach,$37.3M,Empty 2-acre lot,Residential,https://therealdeal.com/miami/2019/07/08/russi...,NaN,"(525 N County Rd, Palm Beach, FL 33480, USA, (...","(26.732162, -80.0367618, 0.0)",26.732162,-80.036762,0.0
3,NaN,NaN,Sergei Adoniev,A former banana importer and co-owner of a maj...,"111 West 57th street, Manhattan, New York",NYC,$21M,Sergei Adoniev and Albert Avdolyan invested in...,Residential,https://therealdeal.com/2018/12/20/the-russian...,NaN,"(111 W 57th St, New York, NY 10019, USA, (40.7...","(40.7648261, -73.97781739999999, 0.0)",40.764826,-73.977817,0.0
4,NaN,NaN,Albert Avdolyan,A wealthy telecommunications magnate. Sergei A...,"111 West 57th street, Manhattan, New York",NYC,$21M,Sergei Adoniev and Albert Avdolyan invested in...,Residential,https://therealdeal.com/2018/12/20/the-russian...,NaN,"(111 W 57th St, New York, NY 10019, USA, (40.7...","(40.7648261, -73.97781739999999, 0.0)",40.764826,-73.977817,0.0
5,No,May 2021,Boris Kuzinez,The CEO and Founder of Five Points Development,"260 Fifth Avenue, Manhattan, New York",NYC,$52.5M,Kuzinez had purchased this property in 2016 an...,Residential,https://therealdeal.com/2021/05/28/with-52m-sa...,NaN,"(260 5th Ave, New York, NY 10001, USA, (40.745...","(40.7451797, -73.9873646, 0.0)",40.745180,-73.987365,0.0
6,No,September 2018,Roman Abramovich,"After the fall of the Soviet Union, Abramovich...","9 East 75th street, Manhattan, New York",NYC,$74M,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(9 E 75th St, New York, NY 10021, USA, (40.774...","(40.77422199999999, -73.96456789999999, 0.0)",40.774222,-73.964568,0.0
7,No,September 2018,Roman Abramovich,"After the fall of the Soviet Union, Abramovich...","15 East 75th street, Manhattan, New York",NYC,16.5M,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(15 E 75th St, New York, NY 10021, USA, (40.77...","(40.774133, -73.9644059, 0.0)",40.774133,-73.964406,0.0
8,No,September 2018,Roman Abramovich,"After the fall of the Soviet Union, Abramovich...","215 East 75th street, Manhattan, New York",NYC,$900K,Abramovich planned to combine the three townho...,Residential,https://nypost.com/2022/02/27/heres-where-russ...,https://therealdeal.com/2016/03/03/dob-denies-...,"(215 E 75th St, New York, NY 10021, USA, (40.7...","(40.7715223, -73.9583525, 0.0)",40.771522,-73.958353,0.0
9,Now owned by relatives,NaN,Oleg Deripaska,"The founder of Rusal, he is blocked from entry...","11 East 64th Street, Manhattan, New York",NYC,$42.5M,NaN,Residential,https://nypost.com/2022/02/27/heres-where-russ...,NaN,"(11 E 64th St, New York, NY 10065, USA, (40.76...","(40.7671909, -73.96962649999999, 0.0)",40.767191,-73.969626,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[3,'building_name']=('87 Park')

In [11]:
df.columns

Index(['Still Own?', 'Sale Date', 'Name', 'Oligarch_Description',
       'Full_Address', 'City', 'Sale Price', 'Property Description',
       'Property Type', 'Story/Reference Link', 'Story/Reference Link #2',
       'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=4)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Russian Oligarch Real Estate Transactions/investments')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    desc = r['Oligarch_Description']
    splitted = textwrap.wrap(desc, 50)
    desc = '<br>'.join(splitted)
    info = (r['Name'],r['Full_Address'],desc)
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Brief Bio: \
    {bold_start}{info[2]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))

m

In [ ]:
m.save('index.html')